# Deep Learning Assignment 5
    Akhil Devarashetti
    M13471127
    01/29/2020
    
### Question:
DL11.py is to do what DL2.py does with an elaborate convolutional network that we also used in class and a quiz. Run it to see if it is any good.
Modify DL11.py according to Aurelien Geron as follows:
    1. Change conv1 to an nn.Conv2d with output channel number 32, reduce the kernel size to 3, and add padding (padding=1).
    2. Change conv2 to an nn.Conv2d with input channel number 32 and out channel number 64, kernel size 3, padding 1, and stride 2.
    3. Remove nn.Dropout2d from both __init__ and forward.
    3. Remove the max_pool2d after conv1.
    4. Change the input and output dimensions for Linears (and view) from 320 to a number matching the flattened output size of F.max_pool2d, and from 50 to 64.
Run this new program and screen shot its execution.

In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
batch_size_train = 256
n_epochs = 10

In [3]:
train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('./data/', train=True, download=True,
                               transform=torchvision.transforms.ToTensor()),
    batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('./data/', train=False, download=True,
                               transform=torchvision.transforms.ToTensor()))

In [4]:
test_size = len(test_loader.dataset)

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)  # Modified conv1
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1, stride=2)  # Modified conv2
        # self.conv2_drop = nn.Dropout2d()  # Removed the dropout layer
        self.fc1 = nn.Linear(3136, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))  # Removed the max_pool2d after conv1.
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 3136)
        x = F.relu(self.fc1(x))
        # x = F.dropout(x, training=self.training)  # Removed the dropout layer
        x = self.fc2(x)
        return x

In [9]:
model3 = Net()
optimizer = optim.SGD(model3.parameters(), 1e-2)
loss_fn = nn.CrossEntropyLoss()

In [10]:
for epoch in range(n_epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        p = model3(data)
        train_loss = loss_fn(p, target)
        if batch_idx % 100 == 0:
            print('train', epoch, batch_idx, float(train_loss))
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
    m = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        if int(torch.argmax(model3(data))) == int(target[0]):
            m = m + 1
    print("test", epoch, m, "among", test_size, "correctly classified")

train 0 0 2.302032709121704
train 0 100 2.2749319076538086
train 0 200 2.1821787357330322
test 0 5284 among 10000 correctly classified
train 1 0 2.119394302368164
train 1 100 1.377989411354065
train 1 200 0.631195604801178
test 1 8450 among 10000 correctly classified
train 2 0 0.48948347568511963
train 2 100 0.46687960624694824
train 2 200 0.36679014563560486
test 2 8822 among 10000 correctly classified
train 3 0 0.36338454484939575
train 3 100 0.4767206609249115
train 3 200 0.3153747618198395
test 3 8828 among 10000 correctly classified
train 4 0 0.4497474133968353
train 4 100 0.30992913246154785
train 4 200 0.4247882068157196
test 4 8957 among 10000 correctly classified
train 5 0 0.44322335720062256
train 5 100 0.32874220609664917
train 5 200 0.2761602997779846
test 5 8980 among 10000 correctly classified
train 6 0 0.469051718711853
train 6 100 0.3495524525642395
train 6 200 0.41422462463378906
test 6 9065 among 10000 correctly classified
train 7 0 0.3360041379928589
train 7 100 0.24

### The new model has an accuracy of 92.98% which is better than 84.65% with previous model